In [532]:
import requests
from bs4 import BeautifulSoup
import random
import pymongo
from bson.objectid import ObjectId

pays_europe = [
    "Albanie",
    "Andorre",
    "Autriche",
    "Belgique",
    "Biélorussie",
    "Bosnie-Herzégovine",
    "Bulgarie",
    "Chypre",
    "Croatie",
    "Danemark",
    "Estonie",
    "Finlande",
    "France",
    "Géorgie",
    "Grèce",
    "Hongrie",
    "Islande",
    "Irlande",
    "Italie",
    "Kosovo",
    "Lettonie",
    "Liechtenstein",
    "Lituanie",
    "Luxembourg",
    "Macédoine",
    "Malte",
    "Moldavie",
    "Monaco",
    "Monténégro",
    "Norvège",
    "Pays-Bas",
    "Pologne",
    "Portugal",
    "République tchèque",
    "Roumanie",
    "Royaume-Uni",
    "Russie",
    "Saint-Marin",
    "Serbie",
    "Slovaquie",
    "Slovénie",
    "Suède",
    "Suisse",
    "Transnistrie",
    "Ukraine",
    "Vatican"
]


links_list=[]
URL='https://tres-ecodesign.com/collections/hubsch?page='
for i in range(1,10):
    #print(URL+str(i))
    
   


    response = requests.get(URL+str(i))
    webpage=requests.get(URL+str(i), headers=HEADERS)
    soup = BeautifulSoup(webpage.content,'lxml')
    links=soup.find_all("a", attrs={"class":'grid-item__link'})
    if response.status_code == 200:
        # La page existe
        # Vérifiez maintenant si le texte "Aucun produit ne correspond à votre recherche" est présent sur la page
        page_content = response.content
        pattern = "Aucun produit ne correspond à votre recherche"
        match = re.search(pattern, page_content.decode('utf-8'))
        if match:
            # Le texte a été trouvé
            1==1
        else:
            
            for link in links:
                links_list.append(link.get('href'))   
            # Le texte n'a pas été trouvé
           
    else:
        # La page n'existe pas ou il y a eu un problème
        print("La page n'existe pas ou il y a eu un problème lors de la récupération de la page.")
    
    
    


        
        

Poidsmeuble=[]
Matière=[]
Provenance=[]
Tapprovisionnement=[]
TDistribution=[]


title_list=[]
user_domicile = []
Dispositif_Durée_de_vie =[]
Dispositif_Fin_de_vie=[]

for i in range(len(links_list)):
    if random.randint(0, 1) == 0:
        user_domicile.append("Marseille")
    else:
        user_domicile.append("Autre")
for link in links_list:
    
    new_webpage = requests.get("https://tres-ecodesign.com" + link, headers=HEADERS)

    new_soup = BeautifulSoup(new_webpage.content, "lxml")
    title=new_soup.find("h1", attrs={"class":'h1 product-single__title'})
    title_text = title.get_text()
    title_list.append(title_text)
    
    
    div = new_soup.find(class_='index-section')
    poids = "Poids"

    # Recherche de l'index du mot dans la chaîne
    index = div.text.find(poids)

    # Vérification de l'existence du mot dans la chaîne
    if index != -1:
        # Extraction des 8 lettres suivantes
        extracted_string = div.text[index+len(poids)+2:index+len(poids)+7]
        extracted_string =''.join([i for i in   extracted_string if i.isdigit() or i==',' and '.'])
        #print(extracted_string)
        Poidsmeuble.append(extracted_string)
        
    else:
        Poidsmeuble.append('/')
    Matériaux="Matériaux"
    # Recherche de l'index du mot dans la chaîne
    index = div.text.find(Matériaux)
    
    # Vérification de l'existence du mot dans la chaîne
    if index != -1:
        # Extraction des 8 lettres suivantes
        extracted_string = div.text[index+len(Matériaux)+2:div.text.find('\n', index)]
        FSC="FSC"
        if FSC in extracted_string:
            Dispositif_Durée_de_vie.append("Réparabilité intégrée + BOIS FSC ")
        else:
            Dispositif_Durée_de_vie.append("Réparabilité intégrée")
        #print(extracted_string)
        Matière.append(extracted_string)
        
    else:
        Matière.append('/')
        
    Origine="Origine"
    # Recherche de l'index du mot dans la chaîne
    index = div.text.find(Origine)
    num=links_list.index(link)
    # Vérification de l'existence du mot dans la chaîne
    if index != -1:
        # Extraction des 8 lettres suivantes
        extracted_string = div.text[index+len(Origine)+2:div.text.find('\n', index)]
        
        #print(extracted_string)
        if extracted_string in pays_europe :
            if extracted_string == "France":
                Provenance.append("Origine France")
                Tapprovisionnement.append("Train/Camion")
                TDistribution.append("Colissimo/Spé")
                
            elif user_domicile[num] == 'Marseille' : 
                TDistribution.append("Vélo Cargo")
                Provenance.append("Origine France")
                Tapprovisionnement.append("Train/Camion")

            else:
                Provenance.append("Provenance Européenne")
                Tapprovisionnement.append("TrainEU/CamionEU/BateauEU")
                TDistribution.append("Colissimo/Spé")
             

                    
        else:
            Provenance.append("Produit hors Europe")
            Tapprovisionnement.append("TrainM/CamionM/BateauM")
            TDistribution.append("Colissimo/Spé")
    
    else:
        Provenance.append('/')
        Tapprovisionnement.append('/')
        TDistribution.append("Colissimo/Spé")
   
    Dispositif_Fin_de_vie.append("Matière recyclable ~")

   

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ma_base_de_donnees"]
collection = db["ma_collection"]




for i in range(len(links_list)):
    document = {
        "Numéro": ObjectId(),
        "Produits": title_list[i],
        "Poids": Poidsmeuble[i],
        "Matière": Matière[i],
        "Transport d'approvisionnement": Tapprovisionnement[i],
        "Lieu de fabrication": Provenance[i],
        "Transport de distribution" : TDistribution[i],
        "Dispositifs supplémentaires augmentant la durée de vie": Dispositif_Durée_de_vie[i],
        " Dispositifs supplémentaires d’accompagnement de la fin de vie": Dispositif_Fin_de_vie[i],
    }
    collection.insert_one(document)





In [533]:
EDispositif_Durée_de_vie=["Réparabilité intégrée","Réparabilité sous certaines conditions","Pas de réparabilité"]
EDispositif_Fin_de_vie=["Matière entièrement recyclable","Matière recyclable ~","Matière non recyclable"]
ETapprovisionnement=["Train/Camion","TrainEU/CamionEU/BateauEU","TrainM/CamionM/BateauM"]
EProvenance=["Origine France","Provenance Européenne","Produit hors Europe"]
ETDistribution=["Vélo Cargo","Colissimo/Spé","Autres"]
Evaluations=[]
Notation=[]
Score=['A','B','C']
for i in range(len(links_list)):
    Evaluation=0
    for durée in range(len(EDispositif_Durée_de_vie)):
        if Dispositif_Durée_de_vie[i] == EDispositif_Durée_de_vie[durée]:
            Evaluation=Evaluation+durée
            #print(Evaluation)
    for fin in range(len(EDispositif_Fin_de_vie)):
        if Dispositif_Fin_de_vie[i] == EDispositif_Fin_de_vie[fin]:
            Evaluation=Evaluation+fin
            #print(Evaluation)
    for appr in range(len(ETapprovisionnement)):
        if Tapprovisionnement[i] == ETapprovisionnement[appr]:
            Evaluation=Evaluation+appr
            #print(Evaluation)
    for origin in range(len(EProvenance)):
        if Provenance[i] == EProvenance[origin]:
            Evaluation=Evaluation+origin
            #print(Evaluation)
    for distrib in range(len(ETDistribution)):
        if TDistribution[i] == ETDistribution[distrib]:
            Evaluation=Evaluation+distrib
            #print(Evaluation)
    Evaluations.append(round(Evaluation/5))

print(Evaluations)
for i in range(len(Evaluations)):
    for score in range(len(Score)):
        if Evaluations[i]==score:
            Notation.append(Score[score])
print(Notation)
            
    
    

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
